In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
import json

# -----------------------------
# PARAMETERS
# -----------------------------
IMG_SIZE = 64
BATCH_SIZE = 16
EPOCHS = 50
NUM_CLASSES = 12

TRAIN_DIR = "dateset/train"
TEST_DIR  = "dateset/test"


train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=5,
    width_shift_range=0.05,
    height_shift_range=0.05,
    zoom_range=0.1
)
test_gen = ImageDataGenerator(rescale=1./255)

train_data = train_gen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode="grayscale",
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

test_data = test_gen.flow_from_directory(
    TEST_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode="grayscale",
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

# -----------------------------
# IMPROVED CNN MODEL
# -----------------------------
model = Sequential([
    Conv2D(32, (3,3), activation="relu", input_shape=(IMG_SIZE, IMG_SIZE, 1)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation="relu"),
    MaxPooling2D(2,2),
    Dropout(0.25),

    Conv2D(128, (3,3), activation="relu"),
    MaxPooling2D(2,2),
    Dropout(0.25),

    Flatten(),
    Dense(128, activation="relu"),
    Dropout(0.3),
    Dense(64, activation="relu"),
    Dense(NUM_CLASSES, activation="softmax")
])

model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

# Early stopping to prevent overfitting
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# -----------------------------
# TRAIN
# -----------------------------
model.fit(
    train_data,
    epochs=EPOCHS,
    validation_data=test_data,
    callbacks=[early_stop]
)

# -----------------------------
# SAVE MODEL
# -----------------------------
model.save("uyir_cnn_model.keras")
print("✅ Model saved as uyir_cnn_model.keras")

# -----------------------------
# SAVE CLASS → LETTER MAPPING
# -----------------------------
class_indices = train_data.class_indices
index_to_letter = {v: k for k, v in class_indices.items()}

with open("uyir_label_map.json", "w", encoding="utf-8") as f:
    json.dump(index_to_letter, f, ensure_ascii=False, indent=2)

print("✅ Label mapping saved as uyir_label_map.json")
print("Label mapping:", index_to_letter)

Found 435 images belonging to 12 classes.
Found 165 images belonging to 12 classes.


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_9 (Conv2D)               │ (None, 62, 62, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 128)            │       589,952 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 12)             │           780 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 691,660 (2.64 MB)

 Trainable params: 691,660 (2.64 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 46s 2s/step - accuracy: 0.0897 - loss: 2.4955 - val_accuracy: 0.0848 - val_loss: 2.4839
Epoch 2/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 44s 2s/step - accuracy: 0.0897 - loss: 2.4838 - val_accuracy: 0.0848 - val_loss: 2.4850
Epoch 3/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 59s 2s/step - accuracy: 0.1057 - loss: 2.4815 - val_accuracy: 0.0848 - val_loss: 2.4832
Epoch 4/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 60s 2s/step - accuracy: 0.0828 - loss: 2.4773 - val_accuracy: 0.0848 - val_loss: 2.4829
Epoch 5/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 57s 2s/step - accuracy: 0.0989 - loss: 2.4738 - val_accuracy: 0.0848 - val_loss: 2.4830
Epoch 6/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 56s 2s/step - accuracy: 0.0828 - loss: 2.4732 - val_accuracy: 0.0848 - val_loss: 2.4818
Epoch 7/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 59s 2s/step - accuracy: 0.0989 - loss: 2.4684 - val_accuracy: 0.0848 - val_loss: 2.4880
Epoch 8/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 60s 2s/step - accuracy: 0.0989 - loss: 2.4648 - val_accuracy: 0.0848 - val_loss:

In [42]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load model
model = load_model("uyir_cnn_model.keras")

# Read image
img = cv2.imread("sample4.jpeg")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Binary for detection
binary = cv2.adaptiveThreshold(
    gray, 255,
    cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
    cv2.THRESH_BINARY_INV,
    11, 2
)

# Find contours
contours, _ = cv2.findContours(
    binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
)

cnt = max(contours, key=cv2.contourArea)
x, y, w, h = cv2.boundingRect(cnt)

# Crop from grayscale (NOT binary)
letter = gray[y:y+h, x:x+w]

# Prepare for CNN
letter = cv2.resize(letter, (64, 64))
letter = letter.astype("float32") / 255.0
letter = letter.reshape(1, 64, 64, 1)

# Predict
pred = model.predict(letter)
pred_index = int(np.argmax(pred))

# Mapping
label_map = {
    0: 'A', 1: 'AA', 2: 'AAE', 3: 'AE', 4: 'AKU',
    5: 'Ai', 6: 'EE', 7: 'I', 8: 'O', 9: 'OO', 10: 'OU', 11: 'U'
}

eng_to_tamil = {
    "A": "அ", "AA": "ஆ", "I": "இ", "EE": "ஈ",
    "U": "உ", "OU": "ஊ", "AE": "எ", "AAE": "ஏ",
    "Ai": "ஐ", "O": "ஒ", "OO": "ஓ", "AKU": "ஔ"
}

tamil_letter = eng_to_tamil[label_map[pred_index]]

print("✅ Predicted Tamil Letter:", tamil_letter)




1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
✅ Predicted Tamil Letter: ஏ


In [43]:

# Load model and test data
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

model = load_model("uyir_cnn_model.keras")

# Load test data
test_gen = ImageDataGenerator(rescale=1./255)
test_data = test_gen.flow_from_directory(
    "dateset/test",
    target_size=(64, 64),
    color_mode="grayscale",
    batch_size=16,
    class_mode="categorical",
    shuffle=False
)

# Evaluate model
loss, accuracy = model.evaluate(test_data)

print(f"✅ Test Loss: {loss:.4f}")
print(f"✅ Test Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")


Found 165 images belonging to 12 classes.
11/11 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.0848 - loss: 2.4818
✅ Test Loss: 2.4818
✅ Test Accuracy: 0.0848 (8.48%)
